In [ ]:
import os
import cv2  # or use PIL if preferred
import numpy as np
from Rpkg.Rfund.InputFeature import InputFeature
from 

def process_image(img):
    """
    Dummy image processing function — replace with your own logic.
    For example: convert to grayscale and apply Gaussian blur.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    return blur

def process_images_in_directory(root_dir):
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                img_path = os.path.join(dirpath, filename)
                print(f"Processing: {img_path}")
                
                # Read image
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Failed to read {img_path}")
                    continue

                # Process image
                processed = process_image(img)

                # Save processed image
                out_path = os.path.join(dirpath, f"processed_{filename}")
                cv2.imwrite(out_path, processed)




ModuleNotFoundError: No module named 'Rpkg'

In [ ]:

# base_path = "/home/eric/Documents/cervicalResearchIIP/img_1006t/featureAug/30aug"  # CHANGE THIS
# process_images_in_directory(base_path)